In [4]:
import numpy as np
import random

In [2]:
def check_xover(p1, p2):
    """
    Check and replace any occurrence of skipped locations in both parents.

    This function checks if the parents skipped a location, and replaces it with
    a 6 (the only location allowed to skip, in our problem) if found.

    Args:
        p1 (list): The first parent.
        p2 (list): The second parent.

    Returns:
        tuple: A tuple containing the modified versions of both parents.
    """

    p1 = [6 if gene == '-' else gene for gene in p1]
    p2 = [6 if gene == '-' else gene for gene in p2]

    return p1, p2

## Cyclic Crossover

In [6]:
def cyclic_xover(p1, p2):
    """
    Perform cyclic crossover between two parents to produce offspring.

    Args:
        p1 (list): The first parent.
        p2 (list): The second parent.

    Returns:
        tuple: A tuple containing two offspring resulting from the cyclic crossover operation.
    """
    cyclic_xover.abbv = 'cx'

    # correcting if there are skips in only one parent
    p1, p2 = check_xover(p1, p2)

    size = len(p1)
    o1 = [0] + [-1] * (size-2) + [0]
    o2 = [0] + [-1] * (size-2) + [0]

    # Choosing the first position, whose value will be saved in start_idx
    start_idx = idx = np.random.randint(1, size-1)
    while True:
        # Assigning the values of the parents to their respective offsprings on that position
        o1[idx] = p1[idx]
        o2[idx] = p2[idx]

        # The next index will be the position on parent1 of the previous value of parent 2
        idx = p1.index(p2[idx])

        # If the new position is equal to the starting index, the cycle is closed,
        # and we break the loop

        if idx == start_idx:
            break

    # Filling the rest of the offsprings with their unlinked parents
    for i in range(size):
        if o1[i] == -1:
            o1[i] = p2[i]
        if o2[i] == -1:
            o2[i] = p1[i]

    return o1, o2

In [8]:
cyclic_xover([0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0], [0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0][::-1])

([0, 1, 8, 9, 5, 2, 4, 3, 6, 7, 0], [0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0])

## PMX Crossover

In [7]:
def pmx_xover(p1, p2):
    """
    Perform partially matched crossover (PMX) between two parents to produce offspring.

    Args:
        p1 (list): The first parent.
        p2 (list): The second parent.

    Returns:
        tuple: A tuple containing two offspring resulting from the PMX crossover operation.
    """
    pmx_xover.abbv = 'pmx'

    # correcting if there are skips in only one parent
    p1, p2 = check_xover(p1, p2)

    size = len(p1)
    # Selecting two crossover points
    xp1, xp2 = sorted(random.sample(range(1, size-1), 2))

    o1 = [0] + [-1] * (size - 2) + [0]
    o2 = [0] + [-1] * (size - 2) + [0]

    # Copying the crossover subset from the parents to their unlinked offsprings
    o1[xp1:xp2+1] = p2[xp1:xp2+1]
    o2[xp1:xp2+1] = p1[xp1:xp2+1]

    # Iterating over the copied range
    for i in range(xp1, xp2+1):
        # if the current parent gene is not on its offspring
        if p1[i] not in o1:
            # we store the position of the current gene of parent 2 on parent1
            idx = p1.index(p2[i])
            # if that position is not empty in the offspring
            while o1[idx] != -1:
                # we go to other position
                idx = p1.index(p2[idx])
            # fill the offspring on that position with the first position of parent 1
            o1[idx] = p1[i]

    # Doing it the same to offspring 2
        if p2[i] not in o2:
            idx = p2.index(p1[i])
            while o2[idx] != -1:
                idx = p2.index(p1[idx])
            o2[idx] = p2[i]

    # Filling the remaining spots with the genes from their parents
    for i in range(size):
        if o1[i] == -1:
            o1[i] = p1[i]
        if o2[i] == -1:
            o2[i] = p2[i]

    return o1, o2

In [9]:
pmx_xover([0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0], [0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0][::-1])

([0, 7, 8, 9, 4, 2, 5, 3, 6, 1, 0], [0, 1, 6, 3, 5, 2, 4, 9, 8, 7, 0])

## Ordered Crossover

In [11]:
def ordered_xover(p1, p2):
    """
    Perform ordered crossover (OX) between two parents to produce offspring.

    Args:
        p1 (list): The first parent.
        p2 (list): The second parent.

    Returns:
        tuple: A tuple containing two offspring resulting from the ordered crossover operation.
    """
    ordered_xover.abbv = 'ox'

    # correcting if there are skips in only one parent
    p1, p2 = check_xover(p1, p2)

    size = len(p1)

    # Selecting two crossover points
    xp1, xp2 = sorted(random.sample(range(1, size-1), 2))

    o1 = [0] + [-1]*(size-2) + [0]
    o2 = [0] + [-1]*(size-2) + [0]

    # Copying the crossover subset from the parents to their unlinked offsprings
    o1[xp1:xp2+1] = p2[xp1:xp2+1]
    o2[xp1:xp2+1] = p1[xp1:xp2+1]

    # Getting a list with the unpasted areas of each parent to their offsprings
    p1_remaining = [gene for gene in p1[1:-1] if gene not in o1]
    p2_remaining = [gene for gene in p2[1:-1] if gene not in o2]

    # We start filling the offspring from the last xover point
    i = xp2+1
    # While there are still areas to fill
    while len(p1_remaining) != 0 and len(p2_remaining) != 0:

        # If we reach the end of our list (individual)
        if i == 11:
          # We go back to the beggining
            i = i % 11

        # If the current position of the offspring is unfilled
        if o1[i] == -1:
          # We fill with the first gene of the remaining from their parents (to preserve the order)
            o1[i] = p1_remaining[0]
            # And remove it from the positions remaining
            p1_remaining.remove(o1[i])

        # The same for the other offspring
        if o2[i] == -1:
            o2[i] = p2_remaining[0]
            p2_remaining.remove(o2[i])

        i += 1

    return o1, o2

In [12]:
ordered_xover([0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0], [0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0][::-1])

([0, 1, 8, 9, 5, 7, 6, 3, 4, 2, 0], [0, 7, 6, 3, 4, 1, 8, 9, 5, 2, 0])

## Two-Point Crossover

In [13]:
def two_point_xover(p1, p2):
    """
    Perform two-point crossover between two parents to produce offspring.

    Args:
        p1 (list): The first parent.
        p2 (list): The second parent.

    Returns:
        tuple: A tuple containing two offspring resulting from the two-point crossover operation.
    """
    two_point_xover.abbv = 'tpx'

    # correcting if there are skips in only one parent
    p1, p2 = check_xover(p1, p2)

    # Choosing crossover points
    start, end = sorted(random.sample(range(1, len(p1)-1), 2))

    # Each offspring will have a subset of their unlinked parent
    o1 = p1[1:start] + p2[start:end] + p1[end:-1]
    o2 = p2[1:start] + p1[start:end] + p2[end:-1]


    # Storing the duplicates
    duplicated1 = list(set([value for value in o1 if o1.count(value) > 1]))
    duplicated2 = list(set([value for value in o2 if o2.count(value) > 1]))

    # Shuffling the order of the lists to guarantee randomness when exchanging the bits between the 2 offsprings
    random.shuffle(duplicated1)
    random.shuffle(duplicated2)

    # Swapping the duplicate values
    for val1, val2 in zip(duplicated1,duplicated2):
        o1[o1.index(val1)] = o2[o2.index(val2)]
        o2[o2.index(val2)] = o1[o1.index(val1)]

    return [0] + o1 + [0] , [0] + o2 + [0]

In [14]:
two_point_xover([0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0], [0, 7, 6, 3, 4, 2, 5, 9, 8, 1, 0][::-1])

([0, 7, 4, 3, 6, 2, 5, 9, 8, 1, 0], [0, 1, 5, 9, 8, 2, 4, 3, 6, 7, 0])